# Navbar

> Responsive navigation bar components with mobile support

In [ ]:
#| default_exp components.navbar

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Tuple, Optional, Any
from fasthtml.common import *
from fasthtml.svg import Svg, Path as SvgPath
from cjm_fasthtml_daisyui.core.testing import create_theme_selector
from cjm_fasthtml_daisyui.components.navigation.navbar import navbar, navbar_start, navbar_end, navbar_center
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_styles, btn_sizes, btn_modifiers
from cjm_fasthtml_daisyui.components.actions.dropdown import dropdown, dropdown_content
from cjm_fasthtml_daisyui.components.navigation.menu import menu, menu_directions
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, stroke_dui
from cjm_fasthtml_daisyui.utilities.border_radius import border_radius
from cjm_fasthtml_tailwind.utilities.effects import shadow
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap, flex_display
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight, text_wrap
from cjm_fasthtml_tailwind.utilities.layout import display_tw, z
from cjm_fasthtml_tailwind.utilities.sizing import h, max_w, w
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_app_core.core.html_ids import AppHtmlIds

## Navigation Link

In [ ]:
#| export
def create_nav_link(
    label:str, # Link text to display
    route, # FastHTML route object with .to() method
    target_id:str=AppHtmlIds.MAIN_CONTENT # HTMX target container ID
) -> FT: # Anchor element with HTMX attributes
    """Create a navigation link with HTMX attributes for SPA-like behavior."""
    return A(
        label,
        href=route.to(),
        hx_get=route.to(),
        hx_target=f"#{target_id}",
        hx_push_url="true",
    )

Creates an anchor element that uses HTMX to load content without full page reload.

In [ ]:
# Example: Create a navigation link
from types import SimpleNamespace

# Mock route object
mock_route = SimpleNamespace(to=lambda: "/about")

link = create_nav_link("About Us", mock_route)
print("Navigation link:")
print(link)

Navigation link:
<a href="/about" hx-get="/about" hx-push-url="true" hx-target="#main-content">About Us</a>


## Navbar Component

In [ ]:
#| export
def create_navbar(
    title:str, # Application title
    nav_items:List[Tuple[str, Any]], # List of (label, route) tuples
    home_route:Optional[Any]=None, # Optional home route for title link
    theme_selector:bool=True, # Whether to include theme selector
    target_id:str=AppHtmlIds.MAIN_CONTENT, # HTMX target container ID
    **navbar_kwargs # Additional kwargs for navbar styling
) -> FT: # Navbar component
    """Create a responsive navigation bar with mobile dropdown menu."""
    # Use first nav item as home if not specified
    if home_route is None and nav_items:
        home_route = nav_items[0][1]
    
    # Hamburger menu icon for mobile
    hamburger_icon = Svg(
        SvgPath(
            stroke_linecap="round",
            stroke_linejoin="round",
            stroke_width="2",
            d="M4 6h16M4 12h8m-8 6h16"
        ),
        xmlns="http://www.w3.org/2000/svg",
        cls=combine_classes(h._5, w._5, stroke_dui.base_content),
        fill="none",
        viewBox="0 0 24 24",
    )
    
    # Mobile dropdown menu items
    mobile_nav_items = Ul(
        *[Li(create_nav_link(label, route, target_id)) for label, route in nav_items],
        tabindex="0",
        cls=combine_classes(
            menu,
            dropdown_content,
            bg_dui.base_100,
            border_radius.box,
            m.t._3,
            w._52,
            p._2,
            shadow.lg
        )
    )
    
    # Build navbar
    navbar_cls = combine_classes(
        navbar,
        bg_dui.base_100,
        shadow.sm,
        p(4)
    )
    
    # Add any custom styling
    if 'cls' in navbar_kwargs:
        navbar_cls = combine_classes(navbar_cls, navbar_kwargs.pop('cls'))
    
    return Div(
        Div(
            # Navbar Start: Logo + Mobile Dropdown
            Div(
                # Mobile dropdown (hidden on lg and above)
                Div(
                    Div(
                        hamburger_icon,
                        tabindex="0",
                        role="button",
                        cls=combine_classes(btn, btn_styles.ghost, btn_modifiers.square)
                    ),
                    mobile_nav_items,
                    cls=combine_classes(dropdown, display_tw.hidden.lg)
                ),
                # Logo/Title
                A(
                    H1(
                        title,
                        cls=combine_classes(
                            font_size.xl,
                            font_size._2xl.md,
                            font_weight.bold,
                            text_dui.base_content,
                            text_wrap,
                            max_w.full
                        )
                    ),
                    href=home_route.to() if home_route else "/",
                    hx_get=home_route.to() if home_route else "/",
                    hx_target=f"#{target_id}",
                    hx_push_url=home_route.to() if home_route else "/",
                    cls=combine_classes(btn, btn_styles.ghost, p(0), max_w.full)
                ) if home_route else H1(
                    title,
                    cls=combine_classes(
                        font_size.xl,
                        font_size._2xl.md,
                        font_weight.bold,
                        text_dui.base_content
                    )
                ),
                cls=combine_classes(navbar_start, gap(2))
            ),
            # Navbar Center: Desktop navigation (hidden on mobile, flex on lg)
            Div(
                Ul(
                    *[Li(create_nav_link(label, route, target_id)) for label, route in nav_items],
                    cls=combine_classes(menu, menu_directions.horizontal, p.x._1)
                ),
                cls=combine_classes(navbar_center, display_tw.hidden, flex_display.lg)
            ) if nav_items else None,
            # Navbar End: Theme selector
            Div(
                create_theme_selector() if theme_selector else None,
                cls=str(navbar_end)
            ) if theme_selector else None,
            cls=navbar_cls,
            **navbar_kwargs
        )
    )

Creates a DaisyUI navbar with:
- Responsive design (mobile dropdown, desktop horizontal menu)
- Optional theme selector
- HTMX-enabled navigation links
- Clickable title that links to home

In [ ]:
# Example: Create a basic navbar
from types import SimpleNamespace

# Mock routes
home = SimpleNamespace(to=lambda: "/")
about = SimpleNamespace(to=lambda: "/about")
settings = SimpleNamespace(to=lambda: "/settings")

navbar = create_navbar(
    title="My FastHTML App",
    nav_items=[
        ("Home", home),
        ("About", about),
        ("Settings", settings)
    ],
    theme_selector=False  # Disable for this example
)

print("Navbar component:")
print(navbar)

Navbar component:
<div><div class="navbar bg-base-100 shadow-sm p-4"><div class="navbar-start gap-2"><div class="dropdown lg:hidden"><div tabindex="0" role="button" class="btn btn-ghost btn-square"><svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" fill="none" class="h-5 w-5 stroke-base-content"><path d="M4 6h16M4 12h8m-8 6h16" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"></path></svg></div><ul tabindex="0" class="menu dropdown-content bg-base-100 rounded-box mt-3 w-52 p-2 shadow-lg"><li><a href="/" hx-get="/" hx-push-url="true" hx-target="#main-content">Home</a></li><li><a href="/about" hx-get="/about" hx-push-url="true" hx-target="#main-content">About</a></li><li><a href="/settings" hx-get="/settings" hx-push-url="true" hx-target="#main-content">Settings</a></li></ul></div><a href="/" hx-get="/" hx-push-url="/" hx-target="#main-content" class="btn btn-ghost p-0 max-w-full"><h1 class="text-xl md:text-2xl font-bold text-base-content text-wrap max-w-full">My 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()